In [3]:
import math
import mpmath

import cvx_opti as opt
import numpy as np
import scipy as sp
import scipy.sparse as spr

n = 5
soln = np.load(f'data/{n}_qubit_soln.npy')
soln = spr.csc_array(soln)
B = spr.load_npz(f'data/{n}_qubit_B.npz')

old_soln, extent, state_vectors, soln = opt.more_precise_soln(n, B, soln, opt.T_state(n))
extent**2, soln

(2.2072246659971833,
 array([[-0.01776695+0.04289322j],
        [ 0.01776695+0.04289322j],
        [ 0.01776695+0.04289322j],
        [ 0.04289322+0.01776695j],
        [ 0.01776695+0.04289322j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.01776695+0.04289322j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.01776695-0.04289322j],
        [ 0.01776695+0.04289322j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.01776695-0.04289322j],
        [ 0.04289322+0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.04289322-0.01776695j],
        [ 0.01776695-0.04289322j],

In [11]:
mpmath.identify(round(extent, 8))

'(2**(88/15)*7**(66/5))/(3**(17/15)*5**(262/15))'

In [8]:
(4/(2 + math.sqrt(2)))**n

2.207224665997184

In [1]:
import mpmath

In [4]:
mpmath.identify(6.23927889)

'(2**(119/41)*7**(138/41))/(3**(238/41)*5**(9/41))'

----------------

In [1]:
import cvx_opti as op
import numpy as np
import scipy as sp
import scipy.sparse as spr

In [2]:
CCZ_state_vecs = np.load('opti_data/CC^2Z_state_vectors.npy')

In [3]:
CCZ_state_vecs.shape

(8, 8)

In [4]:
CCZ = op.CCZ_state(2)
CCZ

array([ 0.35355339,  0.35355339,  0.35355339,  0.35355339,  0.35355339,
        0.35355339,  0.35355339, -0.35355339])

In [5]:
soln = np.linalg.lstsq(CCZ_state_vecs, CCZ, rcond=None)[0]
soln

array([ 1.17756934e-16+0.j,  5.00000000e-01+0.j, -2.54333800e-16+0.j,
        5.00000000e-01+0.j, -3.09844951e-16+0.j,  5.00000000e-01+0.j,
       -2.54333800e-16+0.j, -5.00000000e-01+0.j])

In [6]:
CCZ_state_vecs @ soln

array([ 0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
        0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j, -0.35355339+0.j])

In [65]:
np.linalg.norm(soln, 1)

2.000000000000001

In [66]:
B, extent, old_x = op.optimize_stab_extent(CCZ, 3)

status: optimal
optimal objective value: 1.7777777859107302
(4/(2 + math.sqrt(2)))**n = 1.6080810142133546


In [30]:
num_stab_states = B.shape[0]

c = np.zeros(num_stab_states, dtype=complex)
c[:(1 << 3)] = CCZ
c = spr.csc_array(c, dtype=complex).reshape((num_stab_states, 1))

x_sparse = spr.csc_array(old_x, dtype=complex)
old_soln = c + B @ x_sparse

In [32]:
old_soln = old_soln.toarray()

In [40]:
nz_indices = np.nonzero(old_soln.round(4))[0]
nz_indices

array([568, 584, 632, 649, 696, 714, 764, 779])

In [58]:
old_soln[old_soln.round(4) != 0]

array([0.16666667+1.30878650e-12j, 0.16666667+2.23114833e-11j,
       0.16666667-1.26778215e-11j, 0.16666667+1.03900775e-11j,
       0.16666667-9.58721154e-12j, 0.16666667-4.45943927e-12j,
       0.16666667+4.77460920e-12j, 0.16666667-9.23652376e-12j])

In [38]:
import pickle
with open('data/3_qubit_subgroups.data', 'rb') as reader:
    xmatrs = pickle.load(reader)

In [48]:
from F2_helper.F2_helper import int_to_array
from stabiliser_state.Check_Matrix import Check_Matrix

state_vectors = spr.dok_array((1 << 3, len(nz_indices)), dtype=complex)
for j, index in enumerate(nz_indices):
    basis_num = index // (1 << 3)
    signs = int_to_array(index % (1 << 3), 3)

    sv = Check_Matrix.from_binary_matrix(
        xmatrs[basis_num], signs).get_stabiliser_state().get_state_vector()
    state_vectors[:, j] = sv

In [49]:
state_vectors = state_vectors.toarray()

In [53]:
np.allclose(CCZ_state_vecs, state_vectors)

True

-------

In [1]:
import cvx_opti as op
import numpy as np
import scipy as sp
import scipy.sparse as spr
import pickle

In [8]:
import F2_helper.F2_helper as f2
f2.int_to_array(56, 10)

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0], dtype=int8)

In [14]:
f2.array_to_int(np.array([1, 0, 1, 1, 1, 0, 0, 0]))

184